In [81]:
import numpy as np
import pandas as pd

In [82]:
dataframe = pd.read_csv("piwo.csv", index_col=0, sep=' ')

In [83]:
dataframe

,Koszt,Smak,Procent,Jakość
Imię,,,,
Harnaś,9,2,3,4
Perła,6,3,4,5
Fortuna,5,3,4,5
Heineken,3,4,5,6
Tyskie,7,8,8,8


In [84]:
dataframe.columns.values.tolist()

['Koszt', 'Smak', 'Procent', 'Jakość']

In [85]:
dataframe[["Koszt", "Procent"]]

,Koszt,Procent
Imię,,
Harnaś,9,3
Perła,6,4
Fortuna,5,4
Heineken,3,5
Tyskie,7,8


In [86]:
dataframe["Koszt"]

Imię
Harnaś      9
Perła       6
Fortuna     5
Heineken    3
Tyskie      7
Name: Koszt, dtype: int64

In [87]:
def get_columns_names(dataframe):
    return dataframe.columns.values.tolist()

In [88]:
print(get_columns_names(dataframe))

['Koszt', 'Smak', 'Procent', 'Jakość']


In [89]:
def filter_dataframe_columns(dataframe, columns_to_stay):
    dataframe_columns_names = get_columns_names(dataframe)
    columns_to_drop = set(dataframe_columns_names) - set(columns_to_stay)
    return dataframe.drop(columns=columns_to_drop)

In [122]:
only_good_values = filter_dataframe_columns(dataframe, ["Koszt", "Procent", "Smak"])
only_good_values

,Koszt,Smak,Procent
Imię,,,
Harnaś,9,2,3
Perła,6,3,4
Fortuna,5,3,4
Heineken,3,4,5
Tyskie,7,8,8


In [149]:
def count_eigenvalue(comparison_matrix):
    return np.linalg.eigh(comparison_matrix)

In [150]:
comp_matrix = np.array([[1,2, 9],[1/2,1, 2], [1/9, 1/2, 1]])
print(comp_matrix)
w, v = count_eigenvalue(comp_matrix)
print(w)
print(v)

[[1.         2.         9.        ]
 [0.5        1.         2.        ]
 [0.11111111 0.5        1.        ]]
[0.3462697  0.88888889 1.76484141]
[[ 0.84679693+0.j          0.84679693+0.j          0.84679693-0.j        ]
 [ 0.46601031+0.j         -0.23300515+0.40357677j -0.23300515-0.40357677j]
 [ 0.25645536+0.j         -0.12822768-0.22209685j -0.12822768+0.22209685j]]


In [135]:
def get_final_ranking(dataframe, weights):
    data = dataframe.to_numpy()
    return np.sum(data*weights, axis=-1)

In [136]:
print(make_ranking(only_good_values, w))

[27.05981763-0.166348j 18.02300678-0.166348j 15.01380407-0.166348j
  8.98619593-0.166348j 20.99079729+0.j      ]


In [137]:
def apply_ranking_to_dataframe(dataframe, weigth):
    result_dataframe = dataframe.copy(deep=True)
    ranking = get_final_ranking(dataframe, weigth)
    result_dataframe["Wyniki"] = ranking.tolist()
    result_dataframe = result_dataframe.sort_values("Wyniki", ascending=False)
    return result_dataframe

In [138]:
result_dataframe = apply_ranking_to_dataframe(only_good_values, w)
result_dataframe

,Koszt,Smak,Procent,Wyniki
Imię,,,,
Harnaś,9,2,3,27.059818-0.166348j
Tyskie,7,8,8,20.990797+0.000000j
Perła,6,3,4,18.023007-0.166348j
Fortuna,5,3,4,15.013804-0.166348j
Heineken,3,4,5,8.986196-0.166348j
